In [33]:
import pandas as pd
import numpy as np

dfincomelist = []
for year in range(1998,2007):
    if year < 2001:
        df = pd.read_csv("Variables/Income/Income_{}".format(year), sep = ";",skiprows = 1, names = ["Regiocode","income{}".format(year)], index_col = 0)
        dfincomelist.append(df)
    else:
        df = pd.read_csv("Variables/Income/Income_{}".format(year), sep = ";",skiprows = 0)
        df = df.drop(["ID","SamenstellingVanHetHuishouden"], axis = 1).rename({"RegioS":"Regiocode","GemiddBesteedbInkomenPerHuishouden_2": "income{}".format(year)},axis = 1)
        dfincomelist.append(df)
        
for i,k in enumerate(dfincomelist):
    if i ==0: 
        dfIncome = dfincomelist[0] 
    else:
        dfIncome = dfIncome.merge(dfincomelist[i].set_index("Regiocode"), on=["Regiocode"], how = "outer")

In [34]:
munchange = []
for year in range(1999,2020):
    if year < 2018:
        municipalchanges = pd.read_csv("GI_{}.csv".format(year), sep = ";")  # For every csv file
    else: 
        municipalchanges = pd.read_csv("GI_{}.csv".format(year), sep = ",")  # For every csv file
    ## Drop row provence change (provence change is not necessary for dataframe)
    if year == 2002:
        municipalchanges = municipalchanges.drop([13])
    #Change codes in csv file to match electionresult files
    for i,j in enumerate(municipalchanges["Gemeente Code Old"]):
        if j <= 10:
            municipalchanges.loc[i,"Gemeente Code Old"] = 'G000{}'.format(j)
        elif j <= 100:
            municipalchanges.loc[i,"Gemeente Code Old"] = 'G00{}'.format(j)
        elif j <= 1000:
            municipalchanges.loc[i,"Gemeente Code Old"] = 'G0{}'.format(j)
        else: 
            municipalchanges.loc[i,"Gemeente Code Old"] = 'G{}'.format(j)
    for i,j in enumerate(municipalchanges["Gemeente Code New"]):
        if j <= 100:
            municipalchanges.loc[i,"Gemeente Code New"] = 'G00{}'.format(j)
        elif j <= 1000:
            municipalchanges.loc[i,"Gemeente Code New"] = 'G0{}'.format(j)
        else: 
            municipalchanges.loc[i,"Gemeente Code New"] = 'G{}'.format(j)
    munchange.append(municipalchanges)

In [35]:
for index, code in enumerate(dfIncome["Regiocode"]):
    dfIncome.iloc[index,0] = dfIncome.iloc[index,0].replace("M","")

In [36]:
for Municipalchanges in munchange:
    #     display(Municipalchanges)
    #     Computing old values to new
        regiocodeoldlist = Municipalchanges['Gemeente Code Old'].tolist()
        for i,j in enumerate(dfIncome["Regiocode"]):
            if j in regiocodeoldlist:
                for index, oldcode in enumerate(Municipalchanges["Gemeente Code Old"]):
                    if oldcode == j:
                        dfIncome.iloc[i,0] = Municipalchanges.iloc[index, 2]
    
for year in range(1998,2007):
    dfIncome["Income{}".format(year)] = dfIncome.groupby(["Regiocode"])['income{}'.format(year)].transform('sum')
    dfIncome = dfIncome.drop("income{}".format(year),axis =1 )
    

dfIncome = dfIncome.drop_duplicates(subset=["Regiocode"])


In [37]:
## DATASET VANAF 2007 tot 14

df = pd.read_csv("Variables/Income/Income_2007-14", sep = ";",skiprows = 0)
df = df.drop(["ID","KenmerkenVanHetHuishouden","Populatie"], axis = 1).rename({"RegioS":"Regiocode","GemiddeldBesteedbaarInkomen_3": "income"},axis = 1)

for i,j in enumerate(df["Perioden"]):
    df.iloc[i,1] = j.replace("JJ00","")
    
df = df.pivot(index ='Regiocode', columns ='Perioden')

df.columns = ['_'.join(col) for col in df.columns.values]

df = df.reset_index()
for index, code in enumerate(df["Regiocode"]):
    df.iloc[index,0] = df.iloc[index,0].replace("M","")


In [38]:
for column in df:
    if column == "Regiocode":
        continue
    for index in df.index.values:
        if df.loc[index,column] == "       .":
            df.loc[index,column] = np.NaN
        df.loc[index,column] = float(df.loc[index,column])


In [39]:
for Municipalchanges in munchange:
    #     display(Municipalchanges)
    #     Computing old values to new
        regiocodeoldlist = Municipalchanges['Gemeente Code Old'].tolist()
        for i,j in enumerate(df["Regiocode"]):
            if j in regiocodeoldlist:
                for index, oldcode in enumerate(Municipalchanges["Gemeente Code Old"]):
                    if oldcode == j:
                        df.iloc[i,0] = Municipalchanges.iloc[index, 2]
    
for year in range(2007,2015):
    df["Income{}".format(year)] = df.groupby(["Regiocode"])['income_{}'.format(year)].transform('sum')
    df = df.drop("income_{}".format(year),axis =1 )

df = df.drop_duplicates(subset=["Regiocode"])


In [21]:
dfIncome = dfIncome.merge(df, on = ["Regiocode"], how = "left" )


## SINCE ISLANDS ARE MISSING IN 2007&8, LINEARLY INTERPOLATED.
for column in dfIncome:
    if column == "Regiocode":
        continue
    for index in dfIncome.index.values:
        if dfIncome.loc[index,column] == 0:
            dfIncome.loc[index,column] = np.NaN


dfIncome = dfIncome.set_index("Regiocode")
dfIncome = dfIncome.interpolate(axis = "columns")
dfIncome = dfIncome.reset_index()

In [23]:
for year in range(1998,2012):
    dfIncome["avgincomechange{}".format(year+1)] = (((dfIncome["Income{}".format(year+1)]) - dfIncome["Income{}".format(year)] )/dfIncome["Income{}".format(year)])*100

electionperiods = [[1999,2002,"9802"],[2002,2003,"0203"],[2003,2006,"0306"],[2006,2010,"0610"],[2010,2012,'1012']]
dfIncomeelect12 = pd.DataFrame()


for i in electionperiods:
    startyear = i[0]+1
    avg = dfIncome[["Regiocode","avgincomechange{}".format(startyear)]]
    for j in range(i[0]+2,i[1]+1):
        avg = avg.merge(dfIncome[["avgincomechange{}".format(j),"Regiocode"]],how = "right", on = "Regiocode")
    dfIncomeelect12["avgincchangeelection{}".format(i[2])] = avg.iloc[:,1:].mean(axis = 1)



dfIncomeelect12["Regiocode"] = dfIncome["Regiocode"]



In [26]:
dfIncomeelect12.to_csv('/Users/ruben/Documents/Datascience & Society/Thesis/Variables/Definitiveversions/Income12_version.csv')



In [85]:
df = pd.read_csv("Variables/Income/Income_2012-17", sep = ";",skiprows = 0)

df = df.drop(["ID"], axis = 1).rename({"RegioS":"Regiocode","ParticuliereHuishoudensExclStudenten_122": "income"},axis = 1)

for i,j in enumerate(df["Perioden"]):
    df.iloc[i,1] = j.replace("JJ00","")
    
df = df.pivot(index ='Regiocode', columns ='Perioden')

df.columns = ['_'.join(col) for col in df.columns.values]
display(df)

df = df.reset_index()
for index, code in enumerate(df["Regiocode"]):
    df.iloc[index,0] = df.iloc[index,0].replace("M","")



,income_2012,income_2013,income_2014,income_2015,income_2016,income_2017
Regiocode,,,,,,
GM0003,31.9,32.5,33.7,33.5,34.2,34.7
GM0005,37.1,37.4,39.2,39.1,40.2,41.1
GM0007,33.9,34.5,35.3,34.9,36.9,38.0
GM0009,38.0,38.5,40.2,40.0,41.1,42.7
GM0010,33.0,33.1,34.3,34.5,35.1,36.0
...,...,...,...,...,...,...
GM1969,NaN,NaN,NaN,NaN,NaN,NaN
GM1970,NaN,NaN,NaN,NaN,NaN,NaN
GM1978,NaN,NaN,NaN,NaN,NaN,NaN


,Regiocode,income_2012,income_2013,income_2014,income_2015,income_2016,income_2017
0,G0003,31.9,32.5,33.7,33.5,34.2,34.7
1,G0005,37.1,37.4,39.2,39.1,40.2,41.1
2,G0007,33.9,34.5,35.3,34.9,36.9,38.0
3,G0009,38.0,38.5,40.2,40.0,41.1,42.7
4,G0010,33.0,33.1,34.3,34.5,35.1,36.0
...,...,...,...,...,...,...,...
718,G1969,NaN,NaN,NaN,NaN,NaN,NaN
719,G1970,NaN,NaN,NaN,NaN,NaN,NaN
720,G1978,NaN,NaN,NaN,NaN,NaN,NaN
721,G1979,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
df = df.set_index("Regiocode")
display(df)
df = df.dropna(how = "all")
display(df)
df = df.reset_index()



,income_2012,income_2013,income_2014,income_2015,income_2016,income_2017
Regiocode,,,,,,
G0003,31.9,32.5,33.7,33.5,34.2,34.7
G0005,37.1,37.4,39.2,39.1,40.2,41.1
G0007,33.9,34.5,35.3,34.9,36.9,38.0
G0009,38.0,38.5,40.2,40.0,41.1,42.7
G0010,33.0,33.1,34.3,34.5,35.1,36.0
...,...,...,...,...,...,...
G1969,NaN,NaN,NaN,NaN,NaN,NaN
G1970,NaN,NaN,NaN,NaN,NaN,NaN
G1978,NaN,NaN,NaN,NaN,NaN,NaN


,income_2012,income_2013,income_2014,income_2015,income_2016,income_2017
Regiocode,,,,,,
G0003,31.9,32.5,33.7,33.5,34.2,34.7
G0005,37.1,37.4,39.2,39.1,40.2,41.1
G0007,33.9,34.5,35.3,34.9,36.9,38.0
G0009,38.0,38.5,40.2,40.0,41.1,42.7
G0010,33.0,33.1,34.3,34.5,35.1,36.0
...,...,...,...,...,...,...
G1942,NaN,NaN,NaN,NaN,54.9,57.1
G1945,NaN,NaN,NaN,NaN,40.8,42.8
G1948,NaN,NaN,NaN,NaN,NaN,47.1


In [88]:
munchange = []
for year in range(1999,2020):
    if year < 2018:
        municipalchanges = pd.read_csv("GI_{}.csv".format(year), sep = ";")  # For every csv file
    else: 
        municipalchanges = pd.read_csv("GI_{}.csv".format(year), sep = ",")  # For every csv file
    ## Drop row provence change (provence change is not necessary for dataframe)
    if year == 2002:
        municipalchanges = municipalchanges.drop([13])
    #Change codes in csv file to match electionresult files
    for i,j in enumerate(municipalchanges["Gemeente Code Old"]):
        if j <= 10:
            municipalchanges.loc[i,"Gemeente Code Old"] = 'G000{}'.format(j)
        elif j <= 100:
            municipalchanges.loc[i,"Gemeente Code Old"] = 'G00{}'.format(j)
        elif j <= 1000:
            municipalchanges.loc[i,"Gemeente Code Old"] = 'G0{}'.format(j)
        else: 
            municipalchanges.loc[i,"Gemeente Code Old"] = 'G{}'.format(j)
    for i,j in enumerate(municipalchanges["Gemeente Code New"]):
        if j <= 100:
            municipalchanges.loc[i,"Gemeente Code New"] = 'G00{}'.format(j)
        elif j <= 1000:
            municipalchanges.loc[i,"Gemeente Code New"] = 'G0{}'.format(j)
        else: 
            municipalchanges.loc[i,"Gemeente Code New"] = 'G{}'.format(j)
    munchange.append(municipalchanges)
    

In [89]:
for Municipalchanges in munchange:
    #     display(Municipalchanges)
    #     Computing old values to new
        regiocodeoldlist = Municipalchanges['Gemeente Code Old'].tolist()
        for i,j in enumerate(df["Regiocode"]):
            if j in regiocodeoldlist:
                for index, oldcode in enumerate(Municipalchanges["Gemeente Code Old"]):
                    if oldcode == j:
                        df.iloc[i,0] = Municipalchanges.iloc[index, 2]
    
for year in range(2012,2018):
    df["Income{}".format(year)] = df.groupby(["Regiocode"])['income_{}'.format(year)].transform('sum')
    df = df.drop("income_{}".format(year),axis =1 )

df = df.drop_duplicates(subset=["Regiocode"])
dfIncome = df

In [98]:
dfIncome = dfIncome.reset_index().drop("index",axis = 1)
for year in range(2012,2017):
    dfIncome["avgincomechange{}".format(year+1)] = (((dfIncome["Income{}".format(year+1)]) - dfIncome["Income{}".format(year)] )/dfIncome["Income{}".format(year)])*100

# display(dfIncome)    
electionperiods = [2012,2017,"1217"]
dfIncomeelect12_17 = pd.DataFrame()
regiocodelist =  dfIncome["Regiocode"]




startyear = electionperiods[0]+1
avg = dfIncome[["Regiocode","avgincomechange{}".format(startyear)]]
for j in range(electionperiods[0]+2,electionperiods[1]+1):
    avg = avg.merge(dfIncome[["avgincomechange{}".format(j),"Regiocode"]],how = "right", on = "Regiocode")
dfIncomeelect12_17["avgincchangeelection{}".format(electionperiods[2])] = avg.iloc[:,1:].mean(axis = 1)



dfIncomeelect12_17["Regiocode"] = regiocodelist



In [103]:
dfIncomeelect_98_12 = pd.read_csv("Variables/Definitiveversions/Income12_version.csv", index_col = 0)
display(dfIncomeelect_98_12)
display(dfIncomeelect12_17)

,avgincchangeelection9802,avgincchangeelection0203,avgincchangeelection0306,avgincchangeelection0610,avgincchangeelection1012,Regiocode
0,5.599681,-1.158301e+00,2.065446,2.339684,0.000000,G0003
1,5.813264,-9.933775e-01,2.524962,2.299909,0.426136,G1680
2,5.210921,-1.472627e-14,3.253072,2.097460,0.261476,G1959
3,5.836617,-2.359882e+00,2.965168,2.465197,1.505022,G0358
4,5.755499,-5.208333e-01,-15.428385,2.403509,0.913713,G0197
...,...,...,...,...,...,...
351,5.419050,-1.869159e+00,1.879457,2.002027,0.000000,G0637
352,6.008929,-5.617978e-01,0.976089,2.986322,0.256396,G0638
353,6.272055,-9.433962e-01,2.194912,2.696327,0.672403,G1730
354,5.755612,-9.646302e-01,2.057803,2.132978,0.424127,G0879


,avgincchangeelection1217,Regiocode
0,1.706251,G0003
1,2.117696,G1966
2,2.376959,G1950
3,2.321791,G0014
4,1.762946,G0010
...,...,...
351,2.432349,G1904
352,2.601997,G1911
353,2.211486,G1916
354,2.950600,G1926


In [105]:
dfIncome_election = dfIncomeelect_98_12.merge(dfIncomeelect12_17, on = ["Regiocode"] ,how = "outer")
display(dfIncome_election)
dfIncome_election.to_csv('/Users/ruben/Documents/Datascience & Society/Thesis/Variables/Definitiveversions/Income_election_version.csv')



,avgincchangeelection9802,avgincchangeelection0203,avgincchangeelection0306,avgincchangeelection0610,avgincchangeelection1012,Regiocode,avgincchangeelection1217
0,5.599681,-1.158301e+00,2.065446,2.339684,0.000000,G0003,1.706251
1,5.813264,-9.933775e-01,2.524962,2.299909,0.426136,G1680,2.700639
2,5.210921,-1.472627e-14,3.253072,2.097460,0.261476,G1959,2.357020
3,5.836617,-2.359882e+00,2.965168,2.465197,1.505022,G0358,2.785530
4,5.755499,-5.208333e-01,-15.428385,2.403509,0.913713,G0197,2.383112
...,...,...,...,...,...,...,...
351,5.419050,-1.869159e+00,1.879457,2.002027,0.000000,G0637,2.002689
352,6.008929,-5.617978e-01,0.976089,2.986322,0.256396,G0638,1.930376
353,6.272055,-9.433962e-01,2.194912,2.696327,0.672403,G1730,2.588042
354,5.755612,-9.646302e-01,2.057803,2.132978,0.424127,G0879,2.260963
